DISCLAIMER: 

This Python notebook uses python 3.12.0. Other versions may have errors.

In [23]:
%pip install fastapi uvicorn nest_asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from fastapi import FastAPI
import nest_asyncio
import uvicorn  

nest_asyncio.apply()

app = FastAPI()

@app.get("/")
def read_root():
    return {"Name" : "Luke"}

uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [105772]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57227 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:57227 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57227 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:57228 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [105772]


Slightly more advanced 

In [ ]:
%pip install fastapi uvicorn nest_asyncio python-multipart pandas

In [2]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import pandas as pd
import nest_asyncio
import uvicorn  
import numpy as np
from io import StringIO

nest_asyncio.apply()

data_records = []

app = FastAPI()

@app.on_event("startup")
def load_data():
    global data_records
    df = pd.read_csv("../MLModel/Data/Shapes.csv")
    data_records = df.to_dict(orient="records")

@app.get("/get-data")
async def gather_data():
    return JSONResponse(content=data_records)

uvicorn.run(app, host="127.0.0.1", port=8000)

Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\main.py", line 580, in run
    server.run()
  File "c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\12489\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 31, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\12489\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 93, in run_until_complete
    self._run_once()
  File "C:\Users\12489\AppData\Roaming\Python\Pyt

INFO:     127.0.0.1:57645 - "GET /get-data HTTP/1.1" 200 OK
INFO:     127.0.0.1:57734 - "GET /get-data HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [105772]


Lets go slightly further...

In [1]:
%pip install fastapi uvicorn pandas scikit-learn pydantic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import pandas as pd
import nest_asyncio
import uvicorn  
import numpy as np
from sklearn.tree import DecisionTreeClassifier

df = pd.DataFrame()

app = FastAPI()

model = None
feature_columns = ["Edges","Length","Width"]

nest_asyncio.apply()

class ShapeInput(BaseModel):
    edges: int
    length: int
    height: int

def load_data():
    global df
    df = pd.read_csv("../MLModel/Data/Shapes.csv")
    

def train_model():
    global model
    global df

    X = df[feature_columns]
    y = df["Shape"]

    model = DecisionTreeClassifier()
    model.fit(X, y)

@app.on_event("startup")
def startup():
    load_data()
    train_model()

@app.get("/get-data")
async def gather_data():
    global df
    data_records = df.to_dict(orient="records")
    return JSONResponse(content=data_records)

@app.post("/predict")
def predict_shape(input: ShapeInput):
    input_array = np.array([[input.edges, input.length, input.height]])
    prediction = model.predict(input_array)
    return {"predicted_shape": prediction[0]}

uvicorn.run(app, host="127.0.0.1", port=8000)

Task exception was never retrieved
future: <Task finished name='Task-11' coro=<Server.serve() done, defined at c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\main.py", line 580, in run
    server.run()
  File "c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\12489\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 31, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\12489\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 93, in run_until_complete
    self._run_once()
  File "C:\Users\12489\AppData\Roaming\Python\Py

INFO:     127.0.0.1:58203 - "POST /predict HTTP/1.1" 200 OK


c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


## PYSharp

In [4]:
import numpy as np

data = np.array([1, 2, 3, 4, 5])

result = np.sum(data ** 2)

print(result)

55


More...


In [ ]:
import numpy as np

# Define a structured array with fields: Edges (i4), Length (f8), Width (f8)
shapes = np.array([
    (3, 10.0, 5.0),
    (4, 6.0, 4.0),
    (5, 2.5, 3.0)
], dtype=[('Edges', 'i4'), ('Length', 'f8'), ('Width', 'f8')])

# Compute area: Length * Width
areas = shapes['Length'] * shapes['Width']

# Display results
for i, area in enumerate(areas):
    print(f"Shape {i} - Edges: {shapes['Edges'][i]}, Area: {area}")

In [6]:
from pydantic import BaseModel
from typing import List
import numpy as np

In [8]:
# Alternative - No numpy, just native list
class Shape(BaseModel):
    Edges: int
    Length: float
    Width: float

    def area(self) -> float:
        return self.Length * self.Width

# Create a list of shapes
shapes: List[Shape] = [
        Shape(Edges=3, Length=10.0, Width=5.0),
        Shape(Edges=4, Length=6.0, Width=4.0),
        Shape(Edges=5, Length=2.5, Width=3.0)
    ]

# Compute the area for each shape using a list comprehension
areas = [shape.area() for shape in shapes]

# Optionally convert to NumPy array
area_array = np.array(areas)

# Display results
for i, shape in enumerate(shapes):
    print(f"Shape {i} - Edges: {shape.Edges}, Area: {areas[i]}")

Shape 0 - Edges: 3, Area: 50.0
Shape 1 - Edges: 4, Area: 24.0
Shape 2 - Edges: 5, Area: 7.5
